# Tradeoffs

## Performance vs scalability

- A performant system means your current set up is fast for a given set of users
- A scalable system means that you can easily increase the resources for your set up to serve more users
- In visual format, performance is the current graph value, scalability is the first derivative of the graph. You can have a performant system with shit scalability

- The trade off exists because a single node system can be more performant than a multi-node one due to minimal overhead for coordination. However, single node means you cannot scale components easily

## Latency vs throughput

- Latency is the time taken to perform some action/get some result
- Throughput is how many such actions/results you can get per unit time

- The tradeoff is because maximising throughput can mean handling multiple concurrent tasks, which can lead to resource contention between tasks. This can lead to longer processing time for the average task, but lets you serve more users per unit time

## Availability vs consistency

- CAP Theorem: 3 choose 2
    - Consistency - Every read receives the most recent write or an error
    - Availability - Every request receives a response, without guarantee that it contains the most recent version of the information
    - Partition Tolerance - The system continues to operate despite arbitrary partitioning due to network failures

- Analogy for CAP
    - Imagine you run a note-taking company with your wife. People call in with notes that they want to record, and you record it in your notebook. Then in future, if they want to retrieve the note, they call in again.
        - Obviously, a call will only go to one person. Consequently, this means that any note will only appear in 1 notebook, either yours, or your wife's!
        - This is not good. If someone records a note with your wife, and tries to retrieve it from you, you will not find it!
        - This is a **consistency** problem. Your notebook is not consistent with your wife's

    - To solve the consistency problem, you arrange for a new workflow
        - Whenever someone receives a call, before you hang up the phone, you will tap the other person on the shoulder and write the note in BOTH notebooks
        - If someone doesn't report to work for a span of time, before taking new calls, they will update their notebook with the missing entries.
        - This will ensure consistency, but it also means that while the other person is updating the notebook, you cannot receive a call!
        - This is an **availability** problem.

    - Imagine one day your wife develops a severe flu, and is unable to write or speak to you
        - Now, you cannot receive updates from your wife, even if she wants to update you!
        - You shut down the business for the day until your wife gets better (no **partition tolerance**, if something is unavailable everything gets shut down)
        - Alternatively, you can keep operating (**partition tolerance**), and ignore the lack of updates data from your wife (no **consistency**)
        - Alternatively, you can keep operating (**partition tolerance**), but wait for your wife to come back from the doctor first (no **availability**)

- Consistent + Partition Tolerant
    - Before responding to anyone, wait for all nodes to sync 
    - Therefore, there will be times when the node is not available

- Available + Partition Tolerant
    - Respond immediately with the current version of the data
    - Which can be out of date!

- Consistent + Available
    - This is only theoretical; all distributed systems must be partition tolerant to some extent



### Consistency Patterns

- Follows the analogy above. If you cannot understand this section read the previous one first

- Weak Consistency
    - When you write your note to your wife's notebook, there is no guarantee that it will actually get written in a way that she understands it. 
    - Maybe your handwriting sucks? Maybe you got distracted midway?
    - Weak consistency means you make best effort to write, but without guaranteeing that you can read it

- Eventual consistency
    - To better manage the notebook data between you and your wife, you hire a clerk whose job is to shuttle between the two of you to keep both notebooks in sync
    - Because of this clerk, you can now ignore the job of keeping the 2 notebooks in sync, meaning you can always be available for a call
    - Nonetheless, it also means that for short periods of time, your notebooks can be out of sync (not **consistent**)
    - This is known as **eventual consistency**

- Strong consistency
    - After a write, all reads are guaranteed to see it

### Availability Patterns

- Fail-over
    - To ensure that there is always someone ready to serve the customers, you ensure that there is always someone ELSE ready to do your job
    - Active-Passive
        - You are the main person in your business. But to ensure that the company is staffed when you have food poisoning, you pay your neighbour to be on-call. Your neighbour doesn't do the work in normal times; only when you are sick. Hence he is "passive"
    - Active-Active
        - You hire another person to help you with the business. This person splits the load with you on regular days, and covers you when you are sick.

    - The downside to failover pattern is that there is more hardware to manage
        - if you choose a "passive" failover, there is a risk that your backup needs time to warm up before serving. 
        - if you choose a "passive" failover, there is a risk that your handover is not done properly, so you lose data 


- Replication 
    - To ensure that the data in your notebook is never lost, you can duplicate it
    - Master-slave replication
        - Only YOUR notebook handles new notes (master handles writes)
        - But queries about existing notes can go to any of the replicated notebooks (master+slaves handle reads)
    - Master-master replication
        - You can use either notebook to handle new notes and existing notes
        - But you constantly need to update them to make sure the data in them are consistent

- Availability rules of thumb
    - 99.9% available
        - 8h 45m downtime a year
        - 44m downtime a month
        - 10m downtime a week
        - 1.5m downtime a day
    - 99.99% available
        - 53m downtime a year
        - 4.5m downtime a month
        - 1m downtime a week
        - 10s downtime a day

- Availability Math
    - If components are sequential, total system availability is the product of their availability (because availabiliy is simply probability of request getting processed)
        - i.e. 0.999 * 0.999 = 0.998
    - If components are parallel, total system availability is 1 - product of their failure (because probability of request NOT getting processed is probability of both failing at the same time)
        - 1 - ((1 - 0.999) * (1 - 0.999)) = 0.9999